In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain
import sys
import Graph as G

In [2]:
# Data is from Imdb dataset
# This is a large dataset and can take time to process, especially the edges
data = pd.read_csv ("name.tsv", sep = '\t')
data.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0054452,tt0056404,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050976,tt0050986,tt0060827"


In [12]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = data['knownForTitles'].str.split(',').map(len)

df = pd.DataFrame({'Name': np.repeat(data['primaryName'], lens),
                    'Titles': chainer(data['knownForTitles'])})
df = df[df.Titles != "\\N"]
df = df.sort_values('Titles', ignore_index=True)
df.head(30)

,Name,Titles
0,Carmencita,tt0000001
1,Gaston Paulin,tt0000002
2,Julien Pappé,tt0000003
3,Émile Reynaud,tt0000003
4,Tamara Pappé,tt0000003
5,Claire Lissalde,tt0000003
6,Sophie Nicolle,tt0000003
7,Gaston Paulin,tt0000003
8,Gaston Paulin,tt0000004
9,Charles Kayser,tt0000005


In [13]:
g = G.Graph()
for actor in data['primaryName']:
    g.add_vertex(actor)

In [14]:
unique_df = df["Titles"].unique()
print(unique_df)

['tt0000001' 'tt0000002' 'tt0000003' ... 'tt9916756' 'tt9916764'
 'tt9916856']


In [ ]:
# This will run, it just takes time

current_count = 0
names = []

for i in range(len(df)):
    current_title = unique_df[current_count]
    if df['Titles'][i] != current_title:
        if len(names) > 1:
            for i in range(len(names)):
                for j in range(1+i, len(names)):
                    g.add_edge(names[i], names[j])
        
        current_count = current_count + 1            
        names = []
                       
    names.append(df['Name'][i])

In [48]:
def shortest(graph, start, goal = "Kevin Bacon"):
    explored = []
    queue = [[start]]
    
    if start == goal:
        print("Same Node")
        return
    while queue:
        path = queue.pop(0)
        node = path[-1]
        
        if node not in explored:
            neighbours = graph[node]
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                
                if neighbour == goal:
                    print("Shortest path = ", *new_path)
                    return len(new_path)
            explored.append(node)
    print("No connecting path exists.")

In [49]:
def getInput():
    print("\n To quit the program, type return in answer to a question.")
    name = input("Enter the name of the actor: ")
    if name is "\n":
        quit()
    shortest(g, name)
    getInput()

In [47]:
getInput()


 To quit the program, type return in answer to a question.
Enter the name of the actor: Will Smith


TypeError: 'Graph' object is not subscriptable